$ \frac{1}{4} \sum_{m<n} J_{HS}(m-n)\, \vec{\sigma}_m\cdot\vec{\sigma}_n $ where $J_{HS}(m-n) = \frac{\\pi^2}{N^2 \sin^2 ((m-n)\pi / N)}$, $N$ is number of sites. Use periodic boundary conditions

In [1]:
import numpy as np
import netket as nk
import flax.linen  as nn
from netket.operator.spin import sigmax, sigmay, sigmaz
import matplotlib.pyplot as plt
from src.hamiltonians import wrap_index

In [2]:
def haldane_shastry_model(N):
    hi = nk.hilbert.Spin(s=1/2, N=N)
    sum = 0
    for n in range(N):
        for m in range(n):
            hs = (np.pi**2) / (N**2 * (np.sin((wrap_index(N, n, m))*np.pi/N)**2))
            x = sigmax(hi, m) * sigmax(hi, n)
            y = sigmay(hi, m) * sigmay(hi, n)
            z = sigmaz(hi, m) * sigmaz(hi, n)
            sum += hs * (x+y+z)
    return sum / 4

## Training
Note that the number of hidden features, the samples, and the number of optimization epochs increases with system size.

In [3]:
nqs_en_x = []
nqs_en_y = []
analytical_en_x = []
analytical_en_y = []

In [4]:
for N in range(3, 29):
    if not N % 2 == 0:
        continue
    
    print(f'running: N={N}')

    hs = haldane_shastry_model(N)
    rbm = nk.models.RBM(alpha=1*max(1, N/10), param_dtype=complex, kernel_init=nn.initializers.normal(stddev=0.01))

    hi = nk.hilbert.Spin(s=1/2, N=N)
    sampler = nk.sampler.MetropolisLocal(hi)

    vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=512 + 16 * N)
    optimizer = nk.optimizer.Sgd(learning_rate=0.05)
    preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
    vmc = nk.driver.VMC(hs, optimizer, variational_state=vstate, preconditioner=preconditioner)

    true_energy = -np.pi**2 / 24 * (N + 5 / N) 
    print('expecting energy', true_energy)

    log = nk.logging.RuntimeLog()
    vmc.run(n_iter=250 + 15 * N, out=log)
    opt_energy = vstate.expect(hs)

    nqs_en_x.append(N)
    analytical_en_x.append(N)
    nqs_en_y.append(opt_energy.mean.real)
    analytical_en_y.append(true_energy)

running: N=20
expecting energy -8.327478713419147


100%|██████████| 550/550 [02:09<00:00,  4.25it/s, Energy=-7.4424-0.0074j ± 0.0053 [σ²=0.0231, R̂=1.1319]]


In [6]:
np.savetxt("../data/hs_outputs_4.txt", np.column_stack((nqs_en_x, nqs_en_y, analytical_en_x, analytical_en_y)))